# Imports


In [0]:
import os
import zipfile
import csv
from itertools import islice

Use pip to install PyDrive on your Notebook. 
Further information you can find here: 
https://pythonhosted.org/PyDrive/

In [0]:
# Use pip to install pydrive
!pip install -U -q PyDrive

# Imports Google Drive


In [0]:
# imports 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authorization

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth) # we will make all our drive request with this object 

# Access files of given Directory in Drive

In [0]:
def list_files_of_directory(directory):
    query =  directory + " in parents and trashed=false"
    file_list = drive.ListFile({'q':query}).GetList()
    for file in file_list:
      print('Title: %s, id: %s' % (file['title'], file['id']))

In [0]:
query_directory_root = "\'root\'"
list_files_of_directory(query_directory_root)

In [0]:
query_directory_notebook = "\'your_folder_id\'"
list_files_of_directory(query_directory_notebook)

# Loop through all the files you have in your Google Drive

In [0]:
for file_list in drive.ListFile({'q': 'trashed=true', 'maxResults': 10}):
  print('Received %s files from Files.list()' % len(file_list)) # <= 10
  for file1 in file_list:
      print('title: %s, id: %s' % (file1['title'], file1['id']))

# Load your data into the Google Colab Notebook 

Now that we have our id's and now the id of the directory/file or document we can go ahead and load it into the Notebook. 

In [0]:
def load_file_into_notebook(local_env_name,file_id):
  file_to_load = drive.CreateFile({'id': file_id})
  file_to_load.GetContentFile(local_env_name)
  print("Loaded file with id: " + str(file_id) +" into the notebook!")
  print("It is now accessible through the local environment name "+ str(local_env_name))
  return file_to_load 

In [93]:
file_id = 'file_id_of_csv_file'
title_csv = 'avocado_prices.csv'
load_into_notebook(title_csv,file_id)

GoogleDriveFile({u'mimeType': u'text/csv', u'lastViewedByMeDate': u'2018-09-04T14:35:07.905Z', u'appDataContents': False, u'labels': {u'restricted': False, u'starred': False, u'viewed': True, u'hidden': False, u'trashed': False}, u'explicitlyTrashed': False, u'etag': u'"v8V6DvTBE_bfiGlEO9jLLdDEcEE/MTUzNjA3MTcwNzkwNQ"', u'lastModifyingUserName': u'Stavros Filippidis', u'writersCanShare': True, u'owners': [{u'picture': {u'url': u'https://lh3.googleusercontent.com/-8g3VyRRz0BI/AAAAAAAAAAI/AAAAAAAAABA/ifDHoVUTJqg/s64/photo.jpg'}, u'kind': u'drive#user', u'displayName': u'Stavros Filippidis', u'permissionId': u'02695284206209118823', u'isAuthenticatedUser': True, u'emailAddress': u'stavroswd@gmail.com'}], 'id': u'1-NqBFbxfcE6rSq7j3r7uCPIDzhNuZ6LC', u'lastModifyingUser': {u'picture': {u'url': u'https://lh3.googleusercontent.com/-8g3VyRRz0BI/AAAAAAAAAAI/AAAAAAAAABA/ifDHoVUTJqg/s64/photo.jpg'}, u'kind': u'drive#user', u'displayName': u'Stavros Filippidis', u'permissionId': u'026952842062091188

# Folders 
When trying to load a directory it is not working.
Error: No downloadLink/exportLinks for mimetype found in metadata.  I figured the format is not accepted. Simply convert your file into a zip to load it in again. 

In [0]:
directory_gd_id = 'data.zip_id' 
title_directory = 'data.zip'
load_into_notebook(title_directory,directory_gd_id)

# Access and use your data 
 
Open and print the first 10 rows of the loaded avocado_prices.csv file.

In [106]:
def display_csv_data(name):
   with open(name,'r') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in islice(reader, 10): 
          print(row)
            
local_env_name = "avocado_prices.csv"
display_csv_data(local_env_name)

['', 'Date', 'AveragePrice', 'Total Volume', '4046', '4225', '4770', 'Total Bags', 'Small Bags', 'Large Bags', 'XLarge Bags', 'type', 'year', 'region']
['0', '2015-12-27', '1.33', '64236.62', '1036.74', '54454.85', '48.16', '8696.87', '8603.62', '93.25', '0.0', 'conventional', '2015', 'Albany']
['1', '2015-12-20', '1.35', '54876.98', '674.28', '44638.81', '58.33', '9505.56', '9408.07', '97.49', '0.0', 'conventional', '2015', 'Albany']
['2', '2015-12-13', '0.93', '118220.22', '794.7', '109149.67', '130.5', '8145.35', '8042.21', '103.14', '0.0', 'conventional', '2015', 'Albany']
['3', '2015-12-06', '1.08', '78992.15', '1132.0', '71976.41', '72.58', '5811.16', '5677.4', '133.76', '0.0', 'conventional', '2015', 'Albany']
['4', '2015-11-29', '1.28', '51039.6', '941.48', '43838.39', '75.78', '6183.95', '5986.26', '197.69', '0.0', 'conventional', '2015', 'Albany']
['5', '2015-11-22', '1.26', '55979.78', '1184.27', '48067.99', '43.61', '6683.91', '6556.47', '127.44', '0.0', 'conventional', '20

#Save your Data back in the Cloud. 
Once you have done whatever you wanted to do simply save your data back in the Cloud. 
Since you only can import zipped directories i would recommend if you upload directory to compress them first. 

For this example we will save the sample_data provided by Google Colab. You could also upload other data here.  
Machine Learning Models, preprocess data etc. 

In [0]:
def save_file_to_google_drive(name_in_drive,save_in,local_env_name,mimeType):
  file_to_save = drive.CreateFile({'title':name_in_drive,'mimeType':mimeType,"parents": [{"id": save_in }]})
  file_to_save.SetContentFile(local_env_name)
  file_to_save.Upload()

In [0]:
save_destination = 'root' # use root here or the id of the folder in which you want to save your file 
local_env_name = 'sample_data/california_housing_test.csv' #here we upload the provided sample_data from google colab- upload whatever is in your notebook 
mimeType = 'text/csv'
name_in_drive = "first_saved_file.csv"  
save_file_to_google_drive(name_in_drive,save_in,local_env_name,mimeType)